In this notebook, I choose the parking availability to be the sum of the availability within 1/16 of a mile

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [63]:
blockface_detail = pd.read_csv('../1.collect_data/blockface_detail.csv')

In [64]:
blockface_detail.head()

,Unnamed: 0,blockfacename,location,paidparkingarea,paidparkingsubarea,parkingcategory,parkingspacecount,parkingtimelimitcategory,sideofstreet,sourceelementkey,latitude,longitude
0,0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3346935...",Pioneer Square,Core,Paid Parking,4,120.0,SW,1001,47.602873,-122.334694
1,1,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3345126...",Pioneer Square,Core,Paid Parking,8,120.0,NE,1002,47.602949,-122.334513
2,2,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,"{'type': 'Point', 'coordinates': [-122.3351432...",Commercial Core,Waterfront,Paid Parking,7,120.0,NE,1006,47.603674,-122.335143
3,3,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3366575...",Commercial Core,Waterfront,Paid Parking,5,30.0,SW,1009,47.605018,-122.336658
4,4,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3364474...",Commercial Core,Waterfront,Paid Parking,5,120.0,NE,1010,47.605101,-122.336447


In [65]:
import geopy.distance
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

In [66]:
all_coords = np.array((blockface_detail.latitude.values, blockface_detail.longitude.values)).T

In [67]:
def my_dist(u, v):
    return geopy.distance.distance(u, v).miles

In [68]:
dm = pdist(all_coords, my_dist)

In [69]:
new_dfs = []
data = pd.read_pickle('../1.collect_data/data_files/1hr.pkl').replace([np.inf, -np.inf], np.nan).dropna()

for i, row in enumerate(squareform(dm)):
    close_block_inds = np.where(row < (1/16))
    block_keys = blockface_detail.iloc[close_block_inds].sourceelementkey.values
    
    idx = pd.IndexSlice
    df1 = data.loc[idx[block_keys,:],:]
    
    within_x_occupancy = df1.groupby(level=['OccupancyDateTime']).sum()
    within_x_occupancy['PercentOccupied'] = within_x_occupancy['PaidOccupancy'] / within_x_occupancy['ParkingSpaceCount']
    new_dfs.append(within_x_occupancy.assign(SourceElementKey=blockface_detail.iloc[i].sourceelementkey).set_index('SourceElementKey', append=True).swaplevel(0,1))
    

In [70]:
all_dfs = pd.concat(new_dfs)
all_dfs.to_pickle('../1.collect_data/data_files/1hr_1block-average.pkl')